In [ ]:
from pathlib import Path

from utils.utils import compress
    

def make_data(path):
    compress(path)
    compressed_path = Path(path) / 'compressed'
    dst_path = Path(path) / 'preprocessed'
    
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)

    metadata = get_metadata(compressed_path)
    shape_t = metadata["shape_t"]
    shape_c = metadata["shape_c"]
    shape_z = metadata["shape_z"]
    shape_y = metadata["shape_y"]
    shape_x = metadata["shape_x"]

    filename = os.path.join(dst_path, "data.h5")
    metadata_filename = os.path.join(dst_path, "metadata.json")

    if os.path.isfile(filename):
        os.remove(filename)

    shape = (shape_c, shape_z, shape_y, shape_x)

    writer = TimestampedArrayWriter(None, filename,\
                                    shape, dtype=np.uint8,
                                    groupname=None, compression="gzip",
                                    compression_opts=5)

    times = get_times(compressed_path)
    for t in tqdm(range(shape_t)):
        vol = get_slice(compressed_path, t)
        vol = apply_lut(vol, 0.00015, 0.0625, newtype=np.uint8)
        writer.append_data((times[t], vol))
        
    metadata["dtype"] = "uint8"

    with open(metadata_filename, 'w') as outfile:
        json.dump(metadata, outfile, indent=4)

    writer.close()
   